In [48]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_absolute_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
X = pd.read_pickle('all_features.pickle')
X_train, X_test = X[:25000], X[25000:]

### Create labels

In [15]:
with open('elos.pickle', 'rb') as f:
    elos = np.asarray(pickle.load(f))
    
mean_elos = (elos[::2] + elos[1::2]) / 2
elo_difference = elos[::2] - elos[1::2]
mean_elos.shape, elo_difference.shape

((25000,), (25000,))

### ElasticNet

In [79]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV

mean_model = ElasticNetCV()
mean_model.fit(X_train, mean_elos)

diff_model = ElasticNetCV()
diff_model = diff_model.fit(X_train, elo_difference)

mean_pred = mean_model.predict(X_train)
mean_absolute_error(mean_elos, mean_pred)

185.66743621481817

In [80]:
mean_pred_rf = mean_model.predict(X_train)
elo_difference_pred = diff_model.predict(X_train)
white_elos = mean_pred_rf + elo_difference_pred / 2
black_elos = mean_pred_rf - elo_difference_pred / 2
list(zip(white_elos, black_elos))

[(2249.2314919925225, 2243.6676506909312),
 (2227.9903963783008, 2193.2828150845908),
 (2033.9615834867618, 2334.8482648043437),
 (2277.2396764561295, 2273.304214866645),
 (2219.220324501287, 2111.9765711512637),
 (2294.1129062534751, 2299.6915449698231),
 (2201.1577405304938, 2132.4408065094481),
 (2212.5227636028376, 2244.6629729836586),
 (2301.836840668097, 2295.6606972029199),
 (2247.31409872411, 2201.9231088787105),
 (2236.4762509518455, 2151.4027509016282),
 (2263.7461034808834, 2291.3344406559831),
 (2212.9203662572313, 2252.6382773385826),
 (2283.3781238378569, 2236.6479300139863),
 (2236.1429397075917, 2236.0893274449236),
 (2306.5806857786451, 2315.3001576955799),
 (2217.4879297566549, 2162.5587155499588),
 (2179.3645523984287, 2220.5781756488395),
 (2194.2139247441933, 2231.8027807781978),
 (2270.6543450329791, 2322.4274127412064),
 (2312.6494535216798, 2284.4797795180639),
 (2279.3681581311907, 2318.9247563980257),
 (2173.3581376912584, 2077.2928413567456),
 (2280.110656915

### RandomForest predict mean and elo difference

In [74]:
rf_mean = RandomForestRegressor(n_estimators=500, n_jobs=-1)
rf_mean.fit(X_train, mean_elos)
mean_pred_rf = rf_mean.predict(X_train)
mean_absolute_error(mean_elos, mean_pred_rf)

64.504040520000004

In [75]:
rf_diff = RandomForestRegressor(n_estimators=500, n_jobs=-1)
rf_diff.fit(X_train, elo_difference)
elo_difference_pred = rf_diff.predict(X_train)
mean_absolute_error(elo_difference, elo_difference_pred)

55.105530640000005

### Get back white and black ELOS

In [76]:
mean_pred_rf = rf_mean.predict(X_test)
elo_difference_pred = rf_diff.predict(X_test)
white_elos = mean_pred_rf + elo_difference_pred / 2
black_elos = mean_pred_rf - elo_difference_pred / 2
#list(zip(white_elos, black_elos))

### Make submission

In [159]:
submission = pd.read_csv('sampleSubmission.csv')
submission['WhiteElo'] = white_elos
submission['BlackElo'] = black_elos
submission.head()
#submission.to_csv('submission_rf.csv', index=False)

,Event,WhiteElo,BlackElo
0,25001,2249.231492,2243.667651
1,25002,2227.990396,2193.282815
2,25003,2033.961583,2334.848265
3,25004,2277.239676,2273.304215
4,25005,2219.220325,2111.976571


In [57]:
from sklearn.ensemble import GradientBoostingRegressor 

gbm_mean = GradientBoostingRegressor(n_estimators=10)
gbm_mean.fit(X_train, mean_elos)
mean_pred_gbm = gbm_mean.predict(X_train)
mean_absolute_error(mean_elos, mean_pred_gbm)

183.53123368009545